In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import GridSearchCV

In [2]:
# Code written by Joseph Nelson.

def interpret_dftest(dftest):
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    return dfoutput

In [17]:
#read in columns
df = pd.read_csv('allyears.csv')
df.set_index('yyyymmdd')
df.drop(columns = 'Unnamed: 0', inplace = True)

In [18]:
df.columns

Index(['yyyymmdd', 'lat', 'long', 'station_id', 'acton_id', 'station_name',
       'elevation', 'wteq_amt', 'wteq_med', 'wteq_amt_pct_med',
       'prec_wytd_amt', 'prec_wytd_avg', 'prec_wytd_pctavg',
       'big and little wood basins', 'bitterroot', 'chelan, entiat, wenatchee',
       'clearwater and salmon', 'columbia above methow',
       'deschutes, crooked, john day', 'flathead river basin',
       'grand ronde, powder, burnt, imnaha',
       'henrys fork, teton, willow, blackfoot, portneuf',
       'idaho panhandle region', 'kootenai river in montana', 'lewis, cowlitz',
       'lower clark fork river basin', 'lower columbia, hood river',
       'owyhee malheur', 'raft, goose, salmon falls, bruneau',
       'snake above palisades', 'umatilla, walla walla, willow',
       'upper clark fork river basin', 'weiser, payette, boise',
       'white, green, cedar, skykomish, snoqualmi, baker, skagit',
       'willamette', 'yakima, ahtanum'],
      dtype='object')

In [23]:
df.shape

(8812, 36)

In [19]:
basin_list = ['big and little wood basins', 'bitterroot', 'chelan, entiat, wenatchee',
       'clearwater and salmon', 'columbia above methow',
       'deschutes, crooked, john day', 'flathead river basin',
       'grand ronde, powder, burnt, imnaha',
       'henrys fork, teton, willow, blackfoot, portneuf',
       'idaho panhandle region', 'kootenai river in montana', 'lewis, cowlitz',
       'lower clark fork river basin', 'lower columbia, hood river',
       'owyhee malheur', 'raft, goose, salmon falls, bruneau',
       'snake above palisades', 'umatilla, walla walla, willow',
       'upper clark fork river basin', 'weiser, payette, boise',
       'white, green, cedar, skykomish, snoqualmi, baker, skagit',
       'willamette', 'yakima, ahtanum']

In [21]:
len(basin_list)

23

In [ ]:
sams_df = pd.read_csv